In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject10_2.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject26_1.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject07_2.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject27_2.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject29_2.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject13_1.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/SHA256SUMS.txt
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject35_1.edf
/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/Subject28_2.edf
/kaggle/input/eeg-dataset-during-arith

In [154]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# Step 1: Read data from CSV
file_path = '/kaggle/input/eeg-dataset-during-arithmetic-tasks/eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv'
df = pd.read_csv(file_path)

# Display the first few rows to inspect the data
print(df.head())

# Assuming 'Gender' is a categorical variable in your CSV
categorical_columns = ['Gender']

# Step 2: Encode categorical variables
label_encoders = {}
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

# Step 3: Separate features and target
X = df.drop(columns=['Count quality', 'Subject']).values  # Adjust according to your columns
y = df['Count quality'].values

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Normalize/Standardize numerical features
# Drop non-numeric columns before scaling
numeric_columns = df.drop(columns=['Count quality', 'Subject']).columns
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.astype(np.float32))
X_test = scaler.transform(X_test.astype(np.float32))

# Step 6: Reshape data for CNN (assuming EEGNet-like architecture)
num_time_steps = 8  # Adjust according to your data
num_channels = 1  # Adjust according to your data

X_train_reshaped = X_train.reshape(-1, num_time_steps, num_channels, 1)
X_test_reshaped = X_test.reshape(-1, num_time_steps, num_channels, 1)

# Step 7: Define the model
model = Sequential([
    Conv2D(16, (2, 1), activation='relu', input_shape=(num_time_steps, num_channels, 1)),
    MaxPooling2D((2, 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Step 8: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 9: Define early stopping
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Step 10: Fit the model
history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Step 11: Evaluate the model on test data
_, accuracy = model.evaluate(X_test_reshaped, y_test[:X_test_reshaped.shape[0]])  # Ensure y_test matches X_test_reshaped size

print(f"Test accuracy: {accuracy}")

# Step 12: Optionally, make predictions
predictions = model.predict(X_test_reshaped)

     Subject  Age Gender  Recording year  Number of subtractions  \
0  Subject00   21      F            2011                    9.70   
1  Subject01   18      F            2011                   29.35   
2  Subject02   19      F            2012                   12.88   
3  Subject03   17      F            2010                   31.00   
4  Subject04   17      F            2010                    8.60   

   Count quality  
0              0  
1              1  
2              1  
3              1  
4              0  
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6364 - loss: 0.6157 - val_accuracy: 1.0000 - val_loss: 0.5547
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.7273 - loss: 0.6261 - val_accuracy: 1.0000 - val_loss: 0.5416
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5455 - loss: 0.7196 - val_accuracy: 1.0000 - val_loss: 0.5287
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6364 - loss: 0.6174 - val_accuracy: 1.0000 - val_loss: 0.5173
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5455 - loss: 0.6898 - val_accuracy: 1.0000 - val_loss: 0.5059
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5455 - loss: 0.6287 - val_accuracy: 1.0000 - val_loss: 0.4950
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5455 - loss: 0.7037 - val_accuracy: 1.0000 - val_loss: 0.4854
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.6364 - loss: 0.6439 - val_accuracy: 1.0000 - val_loss: 0.4767
Epoch 9/50
1/